<a href="https://colab.research.google.com/github/aditya16072005/weeds-crops/blob/main/crop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Dropout

In [ ]:
import os
import shutil
import random

base_path = '/content/drive/My Drive/crops_and_weeds'
categories = ['crops', 'weeds']
split_ratio = 0.8  # 80% train, 20% test

for category in categories:
    source_folder = os.path.join(base_path, category)
    train_folder = os.path.join(base_path, 'train', category)
    test_folder = os.path.join(base_path, 'test', category)

    # Create destination directories if they don't exist
    os.makedirs(train_folder, exist_ok=True)
    os.makedirs(test_folder, exist_ok=True)

    files = [f for f in os.listdir(source_folder) if f.lower().endswith('.jpg')]
    random.shuffle(files)
    split_idx = int(len(files) * split_ratio)
    train_files = files[:split_idx]
    test_files = files[split_idx:]

    for f in train_files:
        shutil.move(os.path.join(source_folder, f), os.path.join(train_folder, f))
    for f in test_files:
        shutil.move(os.path.join(source_folder, f), os.path.join(test_folder, f))


In [ ]:
import shutil

# Copy train and test folders from Google Drive to /content
shutil.copytree('/content/drive/My Drive/crops_and_weeds/train', '/content/train', dirs_exist_ok=True)
shutil.copytree('/content/drive/My Drive/crops_and_weeds/test', '/content/test', dirs_exist_ok=True)


In [ ]:
train_ds = keras.utils.image_dataset_from_directory(
    directory='/content/train',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(256, 256)
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory='/content/test',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(256, 256)
)


In [ ]:
len(os.listdir('/content/drive/My Drive/crops_and_weeds/train/crops'))


In [ ]:
len(os.listdir('/content/drive/My Drive/crops_and_weeds/train/weeds'))

In [ ]:
len(os.listdir('/content/drive/My Drive/crops_and_weeds/test/crops'))

In [ ]:
len(os.listdir('/content/drive/My Drive/crops_and_weeds/test/weeds'))

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8,1.2],
    zoom_range=0.2
)
test_datagen = ImageDataGenerator()  # No augmentation


train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/crops_and_weeds/train',
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',

)
validation_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/crops_and_weeds/test',
    target_size=(256, 256),
    shuffle=False,
    batch_size=32,
    class_mode='binary'
)


In [ ]:
#CNN MODEL
model = Sequential([

    Conv2D(32, kernel_size=(3,3), padding='valid', activation='relu', input_shape=(256,256,3)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'),

    Conv2D(64, kernel_size=(3,3), padding='valid', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'),

    Conv2D(128, kernel_size=(3,3), padding='valid', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'),



    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])


In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam

optimizer = Adam(learning_rate=0.0001)  # Set your desired learning rate
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
history= model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=50
)


In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], color='red', label='train')
plt.plot(history.history['val_accuracy'], color='blue', label='validation')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'], color='red', label='train')
plt.plot(history.history['val_loss'], color='blue', label='validation')
plt.legend()
plt.show()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    '/content/test',         # <-- set this to your test data directory
    target_size=(256, 256), # or (256, 256) to match your model input
    batch_size=32,
    class_mode='binary',
    shuffle=False           # Important: do not shuffle for evaluation
)


In [ ]:
import numpy as np

# Get predictions (probabilities)
y_pred_probs = model.predict(test_generator)
# Convert probabilities to class labels (0 or 1)
y_pred = (y_pred_probs > 0.5).astype(int).flatten()


In [ ]:
y_true = test_generator.classes


In [ ]:
print(train_generator.class_indices)


In [ ]:
print(test_generator.class_indices)


In [ ]:
# Calculate class distribution
n_crop = len(os.listdir('/content/train'))  # Update with your path
n_weed = len(os.listdir('/content/test'))  # Update with your path
total = n_crop + n_weed
crop_ratio = n_crop / total

print(f"Crop images: {n_crop}, Weed images: {n_weed}")
print(f"Crop ratio: {crop_ratio:.2f}, Weed ratio: {1-crop_ratio:.2f}")

# Get validation set predictions (use your validation generator)
val_probs = model.predict(validation_generator)
val_true = validation_generator.classes

# Calculate optimal threshold using G-mean
from sklearn.metrics import roc_curve
import numpy as np

fpr, tpr, thresholds = roc_curve(val_true, val_probs)
gmeans = np.sqrt(tpr * (1 - fpr))
optimal_idx = np.argmax(gmeans)
optimal_threshold = thresholds[optimal_idx]

print(f"Optimal threshold: {optimal_threshold:.4f}")

# Apply threshold to predictions
val_preds = (val_probs >= optimal_threshold).astype(int)

# Confusion matrix with new threshold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

cm = confusion_matrix(val_true, val_preds)
print("Confusion Matrix with Optimal Threshold:")
print(cm)

# Plot confusion matrix with matplotlib
labels = ['Crop', 'Weed']
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
disp.plot(cmap=plt.cm.Blues)
plt.title(f'Confusion Matrix with Optimal Threshold: {optimal_threshold:.4f}')
plt.show()
